In [1]:
from PIL import Image, ImageEnhance, ImageOps
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from ipywidgets import interactive, HBox, VBox
from simple_ascii_art import img2ascii_arr, img2ascii_arr_v2
from img_processing import preprocess_img, DITHER_MODES

In [2]:
orig_img = Image.open("imgs/irad_grad.bmp").convert("L")

print(orig_img.size)

(512, 512)


In [3]:
def i_preprocess_img(scale_factor, contrast, brightness, quantize_colors, eq, dither):
    proc_img = preprocess_img(orig_img, scale_factor, contrast, brightness, eq, quantize_colors, dither)

    plt.imshow(proc_img, cmap='gray', vmin=0, vmax=255, interpolation='none')

    plt.figure(figsize=(9, 3))
    plt.xticks([x for x in range(0, len(proc_img.histogram()), 25)])
    plt.bar([x for x in range(0, len(proc_img.histogram()))], proc_img.histogram())
    plt.grid()
    plt.show()

    return proc_img

interactive_preprocess = interactive(i_preprocess_img,
            scale_factor=(0.01, 1, 0.01),
            contrast=(0, 2, 0.01),
            brightness=(0, 2, 0.01),
            eq=(0, 1, 0.01),
            quantize_colors=(1, 255, 1),
            dither=DITHER_MODES)

controls = VBox(interactive_preprocess.children[:-1])
display(HBox((controls, interactive_preprocess.children[-1])))

In [9]:
proc_img = interactive_preprocess.result
proc_img_arr = np.array(proc_img)

In [10]:
ascii_arr = img2ascii_arr(proc_img_arr)

In [11]:
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j],ascii_arr[i][j], sep='', end='')
    print('\n', end='')

      ..........::::::::::::;;::;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;::;;::::::::::..::......  ..    
  ..........::::::::::;;;;;;;;;;;;;;;;!!;;!!!!!!!!!!!!!!!!!!;;!!;;;;;;;;;;;;;;;;::::::::::........    
  ........::::::::::;;::;;;;;;;;!!!!;;!!!!!!!!!!!!!!!!!!;;!!!!!!!!;;!!;;;;;;;;;;::;;::::::::::........
......::::::::::;;;;;;;;;;;;!!!!;;!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!;;;;;;;;;;::::::::..::......
......::..::::;;::;;;;;;!!;;!!!!!!!!!!!!++!!++!!++!!++++++!!++!!!!!!!!!!;;!!;;!!;;;;;;;;::::::::......
....::::::::;;;;;;;;;;!!;;!!!!!!!!++!!++++!!++++++++++++!!++++!!++!!!!!!!!!!!!;;;;;;;;;;;;::::::::::..
..::::::::;;;;;;;;;;!!!!!!!!!!++!!++++++++++++++++++++++++++++++++++++!!++!!!!!!!!;;;;;;;;::;;::::..::
..::::::;;::;;;;;;!!!!!!!!++!!++++++++++++oo++oo++oo++oo++oo++++++++++++!!++!!!!!!!!;;;;;;;;::::::::..
::::::;;;;;;;;;;!!!!!!!!!!++++++++++++oo++oooooooooooooooo++oooo++++++++++!!++!!!!!!!!;;;;;;;;;;::::::
::::::;;;;;;;;!!!!!!!!++++++++++++oooooooooooooooooooooooooooooooo++oo+++

In [12]:
ascii_arr = img2ascii_arr_v2(proc_img_arr)

In [13]:
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

      ..:::~~~+~+++v+vvvvvvvvv+v+++~+~~~::.:.      
   ..:::~~+~++vv>cvcccccccccvccccvcvv+++~+:::: .   
  .:.:~+~++v>vccckckJcJkJkJJ\kJckcccvcv>++++~:::.. 
 :::~:+++vccckcJkJJJJJJJJJJJJJJJJJJk\kcccv+++~:::. 
:::++++vccckkJJJJJJJJJJJJJJJJJJJJJJJJkJccccv+v+:~::
:~+~+vccckkJkJJJJJJJAJAAAA2AAAJJJJJJJJJJ\kccc++++~:
~~++cvcckJkJJJJJJGAGGGGGGGdGGGGG2AAJJJJJJJkccc>v+~:
~++cvck\JJJJJJJGGAGddd#QQQdQd#dGdGA2AJJJJJJJkccv+++
++vcccJJJJJJJGAGGd#dQQQQQQQQQQQQddGGGAJJJJJJkkcc>++
++cccJkJJJJJAGGddQQQQQQQQQQQQQQQQQd#GGAJJJJJJkcccv+
+cvck\JJJJJGAGddQQQQQQQQQ@QQQQQQQQQQGGG2JJJJJJ\ccv+
+>cc\JJJJJJGGGdQQQQQQQ@Q@@@@@QQQQQQQ#dGGJJJJJJ\ccc+
+cccJJJJJJ2AGGQQQQQQQQ@@@@@@@@QQQQQQQGGG2JJJJJJccc+
+ccc\JJJJJ2AG#dQQQQQQ@@@@@@@@@QQQQQQd#GGJJJJJJk\c>v
+>cc\JJJJJ2AGGdQQQQQQQQ@@@@@QQQQQQQQdGGGJJJJJJkcc>v
+>cccJJJJJJJGGGdQQQQQQQQQQQQQQQQQQQGdGGJJJJJJJccc>+
++ccckJJJJJJ2AGGd#QQQQQQQQQQQQQQQ#G#GA2JJJJJJc\c>v+
+++ccck\JJJJJ2A2GG##Q#QQQQQQQQQ##GGG2AJJJJJJ\ccc+++
~++v>cc\JJJJJJJ22GGGG#G#####G#GGGG2AJJJJJJJc\cc+>+:
::++>>cc\kJJ